In [ ]:
import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.callbacks import LearningRateScheduler
import math
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

> In the paper, CNN512 with dropout achieves the highest accuracy 0.841

## *The hyperparameter configuration of CNNs.*

Configuration	Values

Optimizer	SGD

Momentum	0.9

Max Learning rate	1×10−1 in custom CNNs

1×10−2 in EfficientNetB0

Base Learning rate	1×10−4

Mode	triangular

Class weight	auto

Dropout	0.5

Augmentation	20 times (I didn't include augmentation in the following code since we decide not to do it)

## *CNN512* 

Layer	Operator	Layer Details

Input Layer	Zero Padding layer	Padding (2,2)

Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3

Layer 2	Batch Normalization layer	-

Layer 3	Relu layer	-

Layer 4	Max Pooling layer	Pooling size (2,2)

Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3

Layer 6	Batch Normalization layer	-

Layer 7	Relu layer	-

Layer 8	Max Pooling layer	Pooling size (2,2)

Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 10	Batch Normalization layer	-

Layer 11	Relu layer	-

Layer 12	Max Pooling layer	Pooling size (2,2)

Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 14	Batch Normalization layer	-

Layer 15	Relu layer	-

Layer 16	Max Pooling layer	Pooling size (2,2)

Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3

Layer 18	Batch Normalization layer	-

Layer 19	Relu layer	-

Layer 20	Max Pooling layer	Pooling size (2,2)

Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3

Layer 22	Batch Normalization layer	-

Layer 23	Relu layer	-

Layer 24	Max Pooling layer	Pooling size (2,2)

Layer 25	Flatten layer	-

Layer 26	FC layer	Neurons number = 1000

Layer 27	Batch Normalization layer	-

Layer 28	Relu layer	-

Layer 29	FC layer	Neurons number = 500

Layer 30	Batch Normalization layer	-

Layer 31	Relu layer	-

Layer 32	FC layer	With SoftMax activation


In [ ]:
input_shape = (512, 512, 3)  # placeholder
num_classes = 5
epochs = 0  # placeholder
x_train = 0  # placeholder
y_train = 0  # placeholder
x_test = 0  # placeholder
y_test = 0  # placeholder

CNN512 Without dropout

In [ ]:
cnn512_nodropout = models.Sequential(
    [
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3), padding="same"),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        # layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_nodropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn512_nodropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn512_nodropout.evaluate(x_test, y_test)

CNN512 With Dropout

In [ ]:
cnn512_dropout = models.Sequential(
    [
        # Input Layer	Zero Padding layer	Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 3	Relu layer	-
        layers.Activation("relu"),
        # Layer 4	Max Pooling layer	Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 7	Relu layer	-
        layers.Activation("relu"),
        # Layer 8	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 11	Relu layer	-
        layers.Activation("relu"),
        # Layer 12	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 15	Relu layer	-
        layers.Activation("relu"),
        # Layer 16	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17	2D CONV layer	Kernel number = 128, kernel size = 3
        layers.Conv2D(128, kernel_size=(3, 3), padding="same"),
        # Layer 18	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 19	Relu layer	-
        layers.Activation("relu"),
        # Layer 20	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 21	2D CONV layer	Kernel number = 200, kernel size = 3
        layers.Conv2D(200, kernel_size=(3, 3), padding="same"),
        # Layer 22	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 23	Relu layer	-
        layers.Activation("relu"),
        # Layer 24	Max Pooling layer	Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 25	Flatten layer	-
        layers.Flatten(),
        # Layer 26	FC layer	Neurons number = 1000
        layers.Dense(1000),
        # Layer 27	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 28	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 29	FC layer	Neurons number = 500
        layers.Dense(500),
        # Layer 30	Batch Normalization layer	-
        layers.BatchNormalization(),
        # Layer 31	Relu layer	-
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 32	FC layer with Softmax activatio
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn512_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn512_dropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn512_dropout.evaluate(x_test, y_test)

## *CNN299* 

Layer	Operator	Layer Details

Input Layer	Zero Padding layer	Padding (2,2)

Layer 1	2D CONV layer	Kernel number = 32, kernel size = 3

Layer 2	Batch Normalization layer	-

Layer 3	Relu layer	-

Layer 4	Max Pooling layer	Pooling size (2,2)

Layer 5	2D CONV layer	Kernel number = 64, kernel size = 3

Layer 6	Batch Normalization layer	-

Layer 7	Relu layer	-

Layer 8	Max Pooling layer	Pooling size (2,2)

Layer 9	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 10	Batch Normalization layer	-

Layer 11	Relu layer	-

Layer 12	Max Pooling layer	Pooling size (2,2)

Layer 13	2D CONV layer	Kernel number = 96, kernel size = 3

Layer 14	Batch Normalization layer	-

Layer 15	Relu layer	-

Layer 16	Max Pooling layer	Pooling size (2,2)

Layer 17	Flatten layer	-

Layer 18	FC layer	Neurons number = 1000

Layer 19	Batch Normalization layer	-

Layer 20	Relu layer	-

Layer 21	FC layer	Neurons number = 500

Layer 22	Batch Normalization layer	-

Layer 23	Relu layer	-

Layer 24	FC layer	With SoftMax activation


In [ ]:
input_shape = (299, 299, 3)  # placeholder
num_classes = 5

CNN299 Without Dropout

In [ ]:
cnn299_nodropout = models.Sequential(
    [
        # Input Layer Zero Padding layer Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1 2D CONV layer Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 3 Relu layer
        layers.Activation("relu"),
        # Layer 4 Max Pooling layer Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5 2D CONV layer Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 7 Relu layer
        layers.Activation("relu"),
        # Layer 8 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 11 Relu layer
        layers.Activation("relu"),
        # Layer 12 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 15 Relu layer
        layers.Activation("relu"),
        # Layer 16 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17 Flatten Layer -
        layers.Flatten(),
        # Layer 18 FC layer Neurons number = 1000
        layers.Dense(1000),
        # Layer 19 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 20 Relu layer
        layers.Activation("relu"),
        # Layer 21 FC layer Neurons number = 500
        layers.Dense(500),
        # Layer 22 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 23 Relu layer
        layers.Activation("relu"),
        # Layer 24 FC layer with Softmax activation for class predictions
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn299_nodropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn299_nodropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn299_nodropout.evaluate(x_test, y_test)

> CNN299 With Dropout

In [ ]:
cnn299_dropout = models.Sequential(
    [
        # Input Layer Zero Padding layer Padding (2,2)
        layers.ZeroPadding2D(padding=(2, 2), input_shape=input_shape),
        # Layer 1 2D CONV layer Kernel number = 32, kernel size = 3
        layers.Conv2D(32, kernel_size=(3, 3), padding="same"),
        # Layer 2 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 3 Relu layer
        layers.Activation("relu"),
        # Layer 4 Max Pooling layer Pooling size (2,2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 5 2D CONV layer Kernel number = 64, kernel size = 3
        layers.Conv2D(64, kernel_size=(3, 3), padding="same"),
        # Layer 6 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 7 Relu layer
        layers.Activation("relu"),
        # Layer 8 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 9 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 10 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 11 Relu layer
        layers.Activation("relu"),
        # Layer 12 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 13 2D CONV layer Kernel number = 96, kernel size = 3
        layers.Conv2D(96, kernel_size=(3, 3), padding="same"),
        # Layer 14 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 15 Relu layer
        layers.Activation("relu"),
        # Layer 16 Max Pooling layer Pooling size (2, 2)
        layers.MaxPooling2D(pool_size=(2, 2)),
        # Layer 17 Flatten Layer -
        layers.Flatten(),
        # Layer 18 FC layer Neurons number = 1000
        layers.Dense(1000),
        # Layer 19 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 20 Relu layer
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 21 FC layer Neurons number = 500
        layers.Dense(500),
        # Layer 22 Batch Normalization layer
        layers.BatchNormalization(),
        # Layer 23 Relu layer
        layers.Activation("relu"),
        layers.Dropout(0.5),
        # Layer 24 FC layer with Softmax activation for class predictions
        layers.Dense(num_classes, activation="softmax"),
    ]
)

In [ ]:
# base learning rate for custom CNNs
base_learning_rate = 1e-4
# maximum learning rate for custom CNNs
max_learning_rate = 1e-1

# Create an instance of SGD optimizer with initial learning rate
optimizer = SGD(learning_rate=base_learning_rate, momentum=0.9)

cnn299_dropout.compile(
    optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy"]
)

# create class weight
classes = np.unique(y_train)
class_weights = compute_class_weight("balanced", classes=classes, y=y_train)
class_weight_dict = dict(zip(classes, class_weights))


# create triangular schedule
def triangular_schedule(epoch):
    """Triangular learning rate scheduler."""
    cycle_length = 10  # Define the length of a cycle
    cycle = math.floor(1 + epoch / (2 * cycle_length))
    x = abs(epoch / cycle_length - 2 * cycle + 1)
    lr = base_learning_rate + (max_learning_rate - base_learning_rate) * max(0, (1 - x))
    return lr


# When fitting the model, include the learning rate scheduler callback
lr_scheduler = LearningRateScheduler(triangular_schedule)

# fitting the model
cnn299_dropout.fit(
    x_train,
    y_train,
    epochs=epochs,
    class_weight=class_weight_dict,
    callbacks=[lr_scheduler],
)

# evaluating the model
cnn299_dropout.evaluate(x_test, y_test)